In [1]:
import sys
import datetime
import numpy as np

import openmm
from openmm import app
from openmm import unit

In [29]:
field = 'mols/field.xml'
config = 'mols/config.pdb'
#statefile = 'state-eq.xml'

forcefield = app.ForceField(field)
pdb = app.PDBFile(config)

In [30]:
temperature = 300.0*unit.kelvin
pressure = (1.0, 1.0, 0.0)

In [31]:
modeller = app.Modeller(pdb.topology, pdb.positions)

print('#  ', modeller.topology.getNumResidues(), 'molecules', modeller.topology.getNumAtoms(), 'atoms', modeller.topology.getNumBonds(), 'bonds')

#   201 molecules 4704 atoms 5116 bonds


In [32]:
boxvec = modeller.topology.getPeriodicBoxVectors()
print('# Box vectors (nm):')
print('#     x      y      z')
for vec in boxvec:
    print(f'#   {vec.x:6.2f} {vec.y:6.2f} {vec.z:6.2f}')

# Box vectors (nm):
#     x      y      z
#     4.02   0.00   0.00
#    -2.01   3.48   0.00
#     0.00   0.00  12.00


In [ ]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, nonbondedCutoff=12.0*unit.angstrom, constraints=app.HBonds, ewaldErrorTolerance=1.0e-5)

In [34]:
print('# Langevin integrator', temperature)
integrator = openmm.LangevinIntegrator(temperature, 5/unit.picosecond, 1*unit.femtosecond)

# Langevin integrator 300.0 K


In [ ]:
print('#   barostat', pressure)
barostat = openmm.MonteCarloBarostat(pressure, temperature)
#barostat = openmm.MonteCarloAnisotropicBarostat(pressure, temperature, True, True, False, 20)
system.addForce(barostat)

#   barostat (1.0, 1.0, 0.0)


7

In [36]:
platform = openmm.Platform.getPlatformByName('OpenCL')
properties = {'Precision': 'single'}

In [37]:
# force settings before creating Simulation
for i, f in enumerate(system.getForces()):
    f.setForceGroup(i)
    if f.getName() == 'HarmonicBondForce':
        f.setUsesPeriodicBoundaryConditions(True)
    if f.getName() == 'HarmonicAngleForce':
        f.setUsesPeriodicBoundaryConditions(True)
    if f.getName() == 'RBTorsionForce':
        f.setUsesPeriodicBoundaryConditions(True)

In [38]:
sim = app.Simulation(modeller.topology, system, integrator, platform, properties)

sim.context.setPositions(modeller.positions)
#sim.context.setVelocitiesToTemperature(temperature)

In [21]:
#print('# coordinates and velocities from', statefile)
#sim.loadState(statefile)

#print('# coordinates and velocities from restart.chk')
#sim.loadCheckpoint('restart.chk')

#state = sim.context.getState()
#sim.topology.setPeriodicBoxVectors(state.getPeriodicBoxVectors())

In [39]:
platform = sim.context.getPlatform()
print('# platform', platform.getName())
for prop in platform.getPropertyNames():
    print('#  ', prop, platform.getPropertyValue(sim.context, prop))


# platform OpenCL
#   DeviceIndex 0
#   DeviceName Apple M1 Pro
#   OpenCLPlatformIndex 0
#   OpenCLPlatformName Apple
#   Precision single
#   UseCpuPme false
#   DisablePmeStream false


In [40]:
state = sim.context.getState(getEnergy=True)
print('# PotentielEnergy', state.getPotentialEnergy())

# PotentielEnergy 79265.81649800244 kJ/mol


In [41]:
for i, f in enumerate(system.getForces()):
    state = sim.context.getState(getEnergy=True, groups={i})
    print('#  ', f.getName(), state.getPotentialEnergy())

#   HarmonicBondForce 1796.0359709262848 kJ/mol
#   RBTorsionForce -526.887228012085 kJ/mol
#   NonbondedForce 79200.03407975176 kJ/mol
#   LennardJones -9159.679175377236 kJ/mol
#   CMMotionRemover 0.0 kJ/mol
#   HarmonicAngleForce 7036.884822845459 kJ/mol
#   LennardJones14 919.4427790641785 kJ/mol
#   MonteCarloAnisotropicBarostat 0.0 kJ/mol


In [42]:
print("# Minimizing energy...")
sim.minimizeEnergy()

state = sim.context.getState(getEnergy=True)
print('# PotentielEnergy', state.getPotentialEnergy())

for i, f in enumerate(system.getForces()):
    state = sim.context.getState(getEnergy=True, groups={i})
    print('#  ', f.getName(), state.getPotentialEnergy())

# Minimizing energy...
# PotentielEnergy 35062.455169877445 kJ/mol
#   HarmonicBondForce 465.48119163513184 kJ/mol
#   RBTorsionForce -256.03919982910156 kJ/mol
#   NonbondedForce 43185.88368912676 kJ/mol
#   LennardJones -12163.152457237587 kJ/mol
#   CMMotionRemover 0.0 kJ/mol
#   HarmonicAngleForce 4008.1728515625 kJ/mol
#   LennardJones14 -177.89801275730133 kJ/mol
#   MonteCarloAnisotropicBarostat 0.0 kJ/mol


In [43]:
sim.reporters = []
sim.reporters.append(app.StateDataReporter(sys.stdout, 100, step=True, speed=True, temperature=True, separator='\t', totalEnergy=True, potentialEnergy=True, density=True))
#sim.reporters.append(app.PDBReporter('traj.pdb', 100))
sim.reporters.append(app.DCDReporter('traj.dcd', 100))
#sim.reporters.append(app.CheckpointReporter('restart.chk', 10000))

In [44]:

sim.step(10000)

#"Step"	"Potential Energy (kJ/mole)"	"Total Energy (kJ/mole)"	"Temperature (K)"	"Density (g/mL)"	"Speed (ns/day)"
100	40813.72706595465	47010.13917532965	116.8755708005641	0.59698656968713	0
200	44709.766518275545	54800.045815150545	190.32096825437435	0.5965170864665734	33.8
300	46730.39565690333	59468.84292252833	240.27021912088023	0.5954002494770372	35.5
400	48121.87134742335	61899.30787086085	259.867440935544	0.595007234641022	35.6
500	49277.82549432322	63746.63994744822	272.90808263993983	0.594343325671266	35.4
600	49346.794219223026	64547.803984848026	286.71861421456845	0.592515057167816	34.7
700	49475.649687973026	65092.458281723026	294.5613342463413	0.592515057167816	34.6
800	49334.414312480716	65207.991460918216	299.4044548882581	0.5905650505353313	34.8
900	49313.8592295195	65103.4666513945	297.8206335495981	0.5891051561511734	35.1
1000	49572.6087534409	65143.6400034409	293.69789051695795	0.5882702065198103	34.9
1100	49322.21067829564	64800.91477985814	291.95643304389944	0.5864

In [45]:
state = sim.context.getState(getPositions=True, getVelocities=True, getIntegratorParameters=False)
coords = state.getPositions()
sim.topology.setPeriodicBoxVectors(state.getPeriodicBoxVectors())
app.PDBFile.writeFile(sim.topology, coords, open('last.pdb', 'w'))

sim.context.setTime(0)
sim.context.setStepCount(0)
sim.saveState('state-eq.xml')
print('# state saved to state-eq.xml')
#sim.saveState('state-np.xml')
#print('# state saved to state-np.xml')

# state saved to state-eq.xml
